In [1]:
import json
import pandas as pd
import requests
from pandas.io.json import json_normalize #package for flattening json in pandas df


In [2]:
page = 'http://api.nobelprize.org/v1/prize.json' 
response = requests.get(page,timeout = 50)
response


<Response [200]>

In [3]:
nobel_raw = json.loads(response.text)
#nobel_raw = response.json()
#print(nobel_raw)

type(nobel_raw)

dict

In [4]:
nobel_cat = json_normalize(nobel_raw['prizes'])
nobel_cat.head(10)

,category,laureates,overallMotivation,year
0,physics,"[{'id': '960', 'firstname': 'Arthur', 'surname...","""for groundbreaking inventions in the field of...",2018
1,chemistry,"[{'id': '963', 'firstname': 'Frances H.', 'sur...",NaN,2018
2,medicine,"[{'id': '958', 'firstname': 'James P.', 'surna...",NaN,2018
3,peace,"[{'id': '966', 'firstname': 'Denis', 'surname'...",NaN,2018
4,economics,"[{'id': '968', 'firstname': 'William D.', 'sur...",NaN,2018
5,physics,"[{'id': '941', 'firstname': 'Rainer', 'surname...",NaN,2017
6,chemistry,"[{'id': '944', 'firstname': 'Jacques', 'surnam...",NaN,2017
7,medicine,"[{'id': '938', 'firstname': 'Jeffrey C.', 'sur...",NaN,2017
8,literature,"[{'id': '947', 'firstname': 'Kazuo', 'surname'...",NaN,2017
9,peace,"[{'id': '948', 'firstname': 'International Cam...",NaN,2017


In [5]:
nobel_laur = json_normalize(nobel_raw['prizes'], record_path = 'laureates')
nobel_laur.head(10)

,firstname,id,motivation,share,surname
0,Arthur,960,"""for the optical tweezers and their applicatio...",2,Ashkin
1,Gérard,961,"""for their method of generating high-intensity...",4,Mourou
2,Donna,962,"""for their method of generating high-intensity...",4,Strickland
3,Frances H.,963,"""for the directed evolution of enzymes""",2,Arnold
4,George P.,964,"""for the phage display of peptides and antibod...",4,Smith
5,Sir Gregory P.,965,"""for the phage display of peptides and antibod...",4,Winter
6,James P.,958,"""for their discovery of cancer therapy by inhi...",2,Allison
7,Tasuku,959,"""for their discovery of cancer therapy by inhi...",2,Honjo
8,Denis,966,"""for their efforts to end the use of sexual vi...",2,Mukwege
9,Nadia,967,"""for their efforts to end the use of sexual vi...",2,Murad


In [6]:
nobel_cat_laur = json_normalize(nobel_raw['prizes'], record_path = 'laureates',
                              meta = ['category','year'])

nobel_cat_laur.head(10)

,firstname,id,motivation,share,surname,category,year
0,Arthur,960,"""for the optical tweezers and their applicatio...",2,Ashkin,physics,2018
1,Gérard,961,"""for their method of generating high-intensity...",4,Mourou,physics,2018
2,Donna,962,"""for their method of generating high-intensity...",4,Strickland,physics,2018
3,Frances H.,963,"""for the directed evolution of enzymes""",2,Arnold,chemistry,2018
4,George P.,964,"""for the phage display of peptides and antibod...",4,Smith,chemistry,2018
5,Sir Gregory P.,965,"""for the phage display of peptides and antibod...",4,Winter,chemistry,2018
6,James P.,958,"""for their discovery of cancer therapy by inhi...",2,Allison,medicine,2018
7,Tasuku,959,"""for their discovery of cancer therapy by inhi...",2,Honjo,medicine,2018
8,Denis,966,"""for their efforts to end the use of sexual vi...",2,Mukwege,peace,2018
9,Nadia,967,"""for their efforts to end the use of sexual vi...",2,Murad,peace,2018


### How many Nobel prizes were given per category?

In [7]:
nobel_per_cat = nobel_cat_laur['category'].value_counts()
nobel_per_cat

medicine      216
physics       210
chemistry     181
peace         133
literature    114
economics      81
Name: category, dtype: int64

### How many Nobel prizes were given to people called ‘Michael’?

In [8]:
result = (nobel_cat_laur['firstname'] =='Michael').value_counts()
result[1]

3

### What is the smallest relative share of a Nobel prize ever given?

In [16]:
max_share = max(nobel_cat_laur['share'])
rel_share = 1/int(max_share)
rel_share

0.25

### Which laureates were awarded multiple prizes? 

In [10]:
full_names = nobel_cat_laur['firstname'] + ' ' + nobel_cat_laur['surname']
#full_names.head(5)

In [11]:
counts = full_names.value_counts()
multi_winners = counts[counts>1]

print(multi_winners)

Comité international de la Croix Rouge (International Committee of the Red Cross)     3
Frederick Sanger                                                                      2
Office of the United Nations High Commissioner for Refugees (UNHCR)                   2
Marie Curie, née Sklodowska                                                           2
John Bardeen                                                                          2
Linus Carl Pauling                                                                    2
dtype: int64


### Which laureates were awarded prizes in multiple categories?

In [12]:
# add column with full names
nobel_cat_laur_2 = nobel_cat_laur
nobel_cat_laur_2['fullnames'] = full_names
nobel_cat_laur_2.head(10)

,firstname,id,motivation,share,surname,category,year,fullnames
0,Arthur,960,"""for the optical tweezers and their applicatio...",2,Ashkin,physics,2018,Arthur Ashkin
1,Gérard,961,"""for their method of generating high-intensity...",4,Mourou,physics,2018,Gérard Mourou
2,Donna,962,"""for their method of generating high-intensity...",4,Strickland,physics,2018,Donna Strickland
3,Frances H.,963,"""for the directed evolution of enzymes""",2,Arnold,chemistry,2018,Frances H. Arnold
4,George P.,964,"""for the phage display of peptides and antibod...",4,Smith,chemistry,2018,George P. Smith
5,Sir Gregory P.,965,"""for the phage display of peptides and antibod...",4,Winter,chemistry,2018,Sir Gregory P. Winter
6,James P.,958,"""for their discovery of cancer therapy by inhi...",2,Allison,medicine,2018,James P. Allison
7,Tasuku,959,"""for their discovery of cancer therapy by inhi...",2,Honjo,medicine,2018,Tasuku Honjo
8,Denis,966,"""for their efforts to end the use of sexual vi...",2,Mukwege,peace,2018,Denis Mukwege
9,Nadia,967,"""for their efforts to end the use of sexual vi...",2,Murad,peace,2018,Nadia Murad


In [13]:
# use list of multiple winners to find data
ix_multiwinners = nobel_cat_laur_2['fullnames'].isin(multi_winners.index)

multi_nobel = nobel_cat_laur_2[ix_multiwinners]
multi_nobel.sort_values('fullnames')

,firstname,id,motivation,share,surname,category,year,fullnames
591,Comité international de la Croix Rouge (Intern...,482,NaN,2,,peace,1963,Comité international de la Croix Rouge (Intern...
723,Comité international de la Croix Rouge (Intern...,482,NaN,1,,peace,1944,Comité international de la Croix Rouge (Intern...
847,Comité international de la Croix Rouge (Intern...,482,NaN,1,,peace,1917,Comité international de la Croix Rouge (Intern...
423,Frederick,222,"""for their contributions concerning the determ...",4,Sanger,chemistry,1980,Frederick Sanger
623,Frederick,222,"""for his work on the structure of proteins, es...",1,Sanger,chemistry,1958,Frederick Sanger
508,John,66,"""for their jointly developed theory of superco...",3,Bardeen,physics,1972,John Bardeen
636,John,66,"""for their researches on semiconductors and th...",3,Bardeen,physics,1956,John Bardeen
600,Linus Carl,217,NaN,1,Pauling,peace,1962,Linus Carl Pauling
651,Linus Carl,217,"""for his research into the nature of the chemi...",1,Pauling,chemistry,1954,Linus Carl Pauling
868,Marie,6,"""in recognition of her services to the advance...",1,"Curie, née Sklodowska",chemistry,1911,"Marie Curie, née Sklodowska"


In [14]:
multi_nobel_uniqueIDcat = multi_nobel.drop_duplicates(['category','id'])
multi_nobel_uniqueIDcat


,firstname,id,motivation,share,surname,category,year,fullnames
417,Office of the United Nations High Commissioner...,515,NaN,1,,peace,1981,Office of the United Nations High Commissioner...
423,Frederick,222,"""for their contributions concerning the determ...",4,Sanger,chemistry,1980,Frederick Sanger
508,John,66,"""for their jointly developed theory of superco...",3,Bardeen,physics,1972,John Bardeen
591,Comité international de la Croix Rouge (Intern...,482,NaN,2,,peace,1963,Comité international de la Croix Rouge (Intern...
600,Linus Carl,217,NaN,1,Pauling,peace,1962,Linus Carl Pauling
651,Linus Carl,217,"""for his research into the nature of the chemi...",1,Pauling,chemistry,1954,Linus Carl Pauling
868,Marie,6,"""in recognition of her services to the advance...",1,"Curie, née Sklodowska",chemistry,1911,"Marie Curie, née Sklodowska"
917,Marie,6,"""in recognition of the extraordinary services ...",4,"Curie, née Sklodowska",physics,1903,"Marie Curie, née Sklodowska"


In [15]:
unique_ids = multi_nobel_uniqueIDcat['id'].drop_duplicates()

columns = multi_nobel_uniqueIDcat.columns
final_result = pd.DataFrame(columns = columns)

for currID in unique_ids:
    curr_df = multi_nobel_uniqueIDcat[multi_nobel_uniqueIDcat['id'] == currID]
    curr_result = curr_df['id'].value_counts()
    
    if curr_result[0] > 1:
        final_result = final_result.append(curr_df)
        #print(curr_df)

final_result

,firstname,id,motivation,share,surname,category,year,fullnames
600,Linus Carl,217,NaN,1,Pauling,peace,1962,Linus Carl Pauling
651,Linus Carl,217,"""for his research into the nature of the chemi...",1,Pauling,chemistry,1954,Linus Carl Pauling
868,Marie,6,"""in recognition of her services to the advance...",1,"Curie, née Sklodowska",chemistry,1911,"Marie Curie, née Sklodowska"
917,Marie,6,"""in recognition of the extraordinary services ...",4,"Curie, née Sklodowska",physics,1903,"Marie Curie, née Sklodowska"
